## Agreements

In [1]:
import os
import sys
import django
from tqdm.auto import tqdm

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

sys.path.append("..")
os.environ['DJANGO_SETTINGS_MODULE'] = 'news_labelling.settings'
django.setup()

from api.models import Comment, ArticleLabel, CommentLabel, Article
from django.contrib.auth.models import User

Primero, veamos qué usuarios etiquetaron más de 4 artículos...

In [2]:
from django.db.models import Count

users = []
for agg in ArticleLabel.objects.values('user__username').annotate(Count('user__username')):
     users.append(agg['user__username'])   
users.remove("jmperez")
users

['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'gdamill', 'lprats']

In [3]:


base_query = {
    "user__username__in": users
}
ArticleLabel.objects.filter(**base_query).count()

30

¿Cuántos etiquetamos todes?

In [4]:
import pandas as pd

pd.options.display.max_columns = 40


articles_labelled = {label.article_id for label in ArticleLabel.objects.filter(**base_query)}

df = pd.DataFrame({"name":users})
df.set_index("name", inplace=True)

for label in ArticleLabel.objects.filter(**base_query):
    username = label.user.username
    df.loc[username, label.article.title] = int(label.is_interesting)
    
df

,Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria”,Escándalo en España: fallan los tests de coronavirus fabricados en China,El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón,"Luis Novaresio contó cómo será su boda con Braulio Bauab: ""Nos va a casar una rabina""",Thelma Fardin contó cómo sigue la causa contra Juan Darthés: “Brasil le permite la impunidad”
name,,,,,
asilva,1.0,1.0,1.0,1.0,1.0
dvalado,1.0,1.0,1.0,1.0,1.0
fdesanctis,1.0,1.0,1.0,1.0,1.0
gclerici,1.0,1.0,1.0,1.0,1.0
gdamill,1.0,1.0,1.0,1.0,1.0
lprats,1.0,1.0,1.0,1.0,1.0


## Agreements sobre odio

Ok, acá agarremos todos los labels sobre comentarios de todes

In [5]:
article_labels = ArticleLabel.objects.filter(
    user__username__in=users,
)

formatted_labels = []

df_comments = pd.DataFrame({"name": users})

df_comments.set_index("name", inplace=True)

for article_label in article_labels:
    username = article_label.user.username
    for comment_label in article_label.comment_labels.all():
        df_comments.loc[username, comment_label.comment_id] = comment_label.is_hateful

# Saco los que no estén etiquetados por todes
#df_comments = df_comments[df_comments.columns[df_comments.notna().all()]]

df_comments

,333495,333496,333497,333498,333499,333500,333501,333502,333503,333504,333505,333506,333507,333508,333509,333510,333511,333512,333513,333514,...,371947,371948,371949,371950,371951,371952,371953,371954,371955,371956,371957,371958,371959,371960,371961,371962,371963,371964,371965,371966
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
asilva,False,True,False,True,False,False,False,False,True,False,True,True,False,False,False,False,True,False,False,False,...,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False
dvalado,False,True,False,True,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,True,True,False,False,False,True,True,True,True,False,True,True,True,True
fdesanctis,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,True,True,False,False,False,True,True,False,False,False,True,True,True,True
gclerici,False,True,False,True,False,False,True,True,True,False,True,True,False,False,False,False,True,False,True,True,...,False,True,False,True,True,False,True,True,False,True,False,True,False,True,True,False,True,True,True,False
gdamill,False,True,False,True,False,False,True,False,True,False,True,True,False,False,False,False,False,True,True,False,...,True,True,False,False,False,False,True,True,False,False,False,False,True,True,True,False,True,True,True,True
lprats,False,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,True,True,False,True,False,True,False,True,True,False,True,True,True,True


In [6]:
from nltk import agreement

def kappa_of(df, subset=None):
    if subset:
        coders = df.loc[subset]
    else:
        coders = df
    
    coders = coders[coders.columns[coders.notna().all()]]
    
    coders = coders.values
    formatted_labels = []
    for i in range(coders.shape[0]):
        # Acá las ponemos siguiendo el siguiente formato
        # [num etiquetador, num instancia, label]
        for j in range(coders.shape[1]):
            formatted_labels.append((i, j, coders[i, j]))


    ratingtask = agreement.AnnotationTask(data=formatted_labels)

    return ratingtask.kappa(), coders.shape[1]


for subset in [
    users,
    ]:
    name = ' '.join(subset)
    kappa, instances = kappa_of(df_comments, subset)
    print(f"Fleiss K para {name:<40} = {kappa:.3f} (#instancias = {instances})")
    
print("\n\nPares\n\n")

for i in range(len(users)):
    for j in range(i+1, len(users)):
        subset = [users[i], users[j]]
        name = ' '.join(subset)
        kappa, instances = kappa_of(df_comments, subset)
        print(f"Fleiss K para {name:<40} = {kappa:.3f} (#instancias = {instances})")


Fleiss K para asilva dvalado fdesanctis gclerici gdamill lprats = 0.595 (#instancias = 241)


Pares


Fleiss K para asilva dvalado                           = 0.547 (#instancias = 241)
Fleiss K para asilva fdesanctis                        = 0.598 (#instancias = 241)
Fleiss K para asilva gclerici                          = 0.544 (#instancias = 241)
Fleiss K para asilva gdamill                           = 0.532 (#instancias = 241)
Fleiss K para asilva lprats                            = 0.546 (#instancias = 241)
Fleiss K para dvalado fdesanctis                       = 0.688 (#instancias = 241)
Fleiss K para dvalado gclerici                         = 0.591 (#instancias = 241)
Fleiss K para dvalado gdamill                          = 0.656 (#instancias = 241)
Fleiss K para dvalado lprats                           = 0.636 (#instancias = 241)
Fleiss K para fdesanctis gclerici                      = 0.514 (#instancias = 241)
Fleiss K para fdesanctis gdamill                       = 0.588 (#ins

In [7]:
print("Cantidad de mensajes etiquetados como odiosos")
df_comments.sum(axis=1)

Cantidad de mensajes etiquetados como odiosos


name
asilva         84.0
dvalado       109.0
fdesanctis    100.0
gclerici      114.0
gdamill       139.0
lprats        123.0
dtype: float64

## Agreements sobre cada categoría

In [8]:
idx = pd.MultiIndex.from_product([CommentLabel.type_mapping, users],
                                names=["categoria", "etiquetador"])

df_labels = pd.DataFrame(index=idx)

df_labels

Empty DataFrame
Columns: []
Index: [(MUJER, asilva), (MUJER, dvalado), (MUJER, fdesanctis), (MUJER, gclerici), (MUJER, gdamill), (MUJER, lprats), (LGBTI, asilva), (LGBTI, dvalado), (LGBTI, fdesanctis), (LGBTI, gclerici), (LGBTI, gdamill), (LGBTI, lprats), (RACISMO, asilva), (RACISMO, dvalado), (RACISMO, fdesanctis), (RACISMO, gclerici), (RACISMO, gdamill), (RACISMO, lprats), (POBREZA, asilva), (POBREZA, dvalado), (POBREZA, fdesanctis), (POBREZA, gclerici), (POBREZA, gdamill), (POBREZA, lprats), (POLITICA, asilva), (POLITICA, dvalado), (POLITICA, fdesanctis), (POLITICA, gclerici), (POLITICA, gdamill), (POLITICA, lprats), (DISCAPACIDAD, asilva), (DISCAPACIDAD, dvalado), (DISCAPACIDAD, fdesanctis), (DISCAPACIDAD, gclerici), (DISCAPACIDAD, gdamill), (DISCAPACIDAD, lprats), (ASPECTO, asilva), (ASPECTO, dvalado), (ASPECTO, fdesanctis), (ASPECTO, gclerici), (ASPECTO, gdamill), (ASPECTO, lprats), (CRIMINAL, asilva), (CRIMINAL, dvalado), (CRIMINAL, fdesanctis), (CRIMINAL, gclerici), (CRIMINAL, gdamill), (CRIMINAL, lprats), (OTROS, asilva), (OTROS, dvalado), (OTROS, fdesanctis), (OTROS, gclerici), (OTROS, gdamill), (OTROS, lprats)]

In [46]:
set([a.article.id for a in article_labels])

{8303, 8473, 8694, 8766, 9122}

In [47]:

article_labels = ArticleLabel.objects.filter(
    user__username__in=users,
)


for article_label in article_labels:
    username = article_label.user.username
    for comment_label in article_label.comment_labels.all():
        for name, field_name in CommentLabel.type_mapping.items():
            df_labels.loc[(name, username), comment_label.comment_id] = getattr(comment_label, field_name)

#df_labels

In [48]:
print("##", ' - '.join(users), '\n')

#users = ["mkondra", "jmperez"]

for category in CommentLabel.type_mapping:
    try:
        kappa, instances = kappa_of(df_labels.loc[category].loc[users])
        print(f"| {category:<12}   |  {kappa:.3f}")
    except ZeroDivisionError as e:
        print(f"No hay elementos para calcular Kappa de {category} - salteando")

print(f"Instancias: {instances}")


## asilva - dvalado - fdesanctis - gclerici - gdamill - lprats 

| MUJER          |  0.506
| LGBTI          |  0.753
| RACISMO        |  0.648
No hay elementos para calcular Kappa de POBREZA - salteando
| POLITICA       |  0.385
| DISCAPACIDAD   |  0.320
| ASPECTO        |  0.531
| CRIMINAL       |  0.888
No hay elementos para calcular Kappa de OTROS - salteando
Instancias: 241


## Alpha Krippendorff

In [49]:
import krippendorff

users = list(df_labels.loc["MUJER"].index)

user_combinations = [users]
user_combinations += [[u1, u2] for u1, u2 in set((u1, u2) for u1 in users for u2 in users if u1 != u2 and u1 < u2)]

for user_list in user_combinations:
    print("="*80 + '\n' * 2)
    print(" - ".join(user_list), "\n")
    for category in CommentLabel.type_mapping:
        mat = df_labels.loc[category].loc[user_list].values
        mat = mat.astype(float)
        alpha = krippendorff.alpha(mat)
        print(f"| {category:<12}   |  {alpha:.3f}")





asilva - dvalado - fdesanctis - gclerici - gdamill - lprats 

| MUJER          |  0.516
| LGBTI          |  0.756
| RACISMO        |  0.667
| POBREZA        |  0.399
| POLITICA       |  0.427
| DISCAPACIDAD   |  0.369
| ASPECTO        |  0.527
| CRIMINAL       |  0.891
| OTROS          |  -0.001


dvalado - gdamill 

| MUJER          |  0.614
| LGBTI          |  0.808
| RACISMO        |  0.759
| POBREZA        |  1.000
| POLITICA       |  0.836
| DISCAPACIDAD   |  0.661
| ASPECTO        |  0.772
| CRIMINAL       |  0.915
| OTROS          |  nan


fdesanctis - lprats 

| MUJER          |  0.662
| LGBTI          |  0.829
| RACISMO        |  0.525
| POBREZA        |  nan
| POLITICA       |  0.389
| DISCAPACIDAD   |  -0.008
| ASPECTO        |  0.539
| CRIMINAL       |  0.957
| OTROS          |  nan


asilva - fdesanctis 

| MUJER          |  0.521
| LGBTI          |  0.787
| RACISMO        |  0.519
| POBREZA        |  0.000
| POLITICA       |  0.497
| DISCAPACIDAD   |  0.663
| ASPECTO   

/home/jmperez/.local/share/virtualenvs/hatespeech-news-labelling-YFKq_jFG/lib/python3.8/site-packages/krippendorff/krippendorff.py:273: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (o * d).sum() / (e * d).sum()


| MUJER          |  0.589
| LGBTI          |  0.815
| RACISMO        |  0.773
| POBREZA        |  0.000
| POLITICA       |  0.539
| DISCAPACIDAD   |  -0.011
| ASPECTO        |  0.620
| CRIMINAL       |  0.945
| OTROS          |  nan


dvalado - gclerici 

| MUJER          |  0.551
| LGBTI          |  0.723
| RACISMO        |  0.866
| POBREZA        |  0.000
| POLITICA       |  0.484
| DISCAPACIDAD   |  -0.008
| ASPECTO        |  0.696
| CRIMINAL       |  0.804
| OTROS          |  nan


gclerici - gdamill 

| MUJER          |  0.683
| LGBTI          |  0.707
| RACISMO        |  0.785
| POBREZA        |  0.000
| POLITICA       |  0.617
| DISCAPACIDAD   |  -0.011
| ASPECTO        |  0.584
| CRIMINAL       |  0.756
| OTROS          |  nan


asilva - gclerici 

| MUJER          |  0.317
| LGBTI          |  0.670
| RACISMO        |  0.648
| POBREZA        |  0.000
| POLITICA       |  0.209
| DISCAPACIDAD   |  -0.004
| ASPECTO        |  0.289
| CRIMINAL       |  0.852
| OTROS          |  -0.0




asilva - fdesanctis - gdamill - lprats 

| Clase          | Acuerdo | 
|----------------|---------|
| MUJER          |  0.501  |
| LGBTI          |  0.772  |
| RACISMO        |  0.635  |
| POBREZA        |  0.499  |
| POLITICA       |  0.436  |
| DISCAPACIDAD   |  0.462  |
| ASPECTO        |  0.517  |
| CRIMINAL       |  0.931  |
| OTROS          |  -0.002 |


Apreciaciones generales:

- CRIMINAL: altísimo acuerdo
- MUJER: acuerdo razonable, pero hay que mejorar
- LGBTI y RACISMO: acuerdo relativamente alto
- POBREZA: ver qué pasa acá (ver Gabriel y Alan que etiquetaron lo mismo)
- POLITICA: esperable bajo acuerdo
- ASPECTO: ver acá qué pasó
- DISCAPACIDAD: ver también
- OTROS: ¿qué marcaron?




## MUJER


In [50]:
from collections import defaultdict

def no_null_columns(df):
    return df[df.columns[df.notna().all()]]

            
def show_from_df(df, show=False):
    article_to_comments = defaultdict(list)
    
    for comment_id in df.columns:
        comment = Comment.objects.get(id=comment_id)
        article_to_comments[comment.article_id].append(comment)
        
    for article_id, comments in article_to_comments.items():
        print("="*45 + "\n")
        article = comments[0].article
        print(article.title, "\n")
        
        for comment in sorted(comments, key=lambda c: sum(df[c.id])):
            print("")
            print("  - ",comment.text, end="")
            if show:
                yes = [annotator for annotator, v in df[comment.id].items() if v] 
                no = [annotator for annotator, v in df[comment.id].items() if not v] 
                print(f"\n     - SI: {yes}  NO: {no}")
            else:
                print("")

In [51]:
def get_agrees_and_disagrees(df_labels, category, users):    
    category_df = df_labels.loc[category].loc[users]

    category_df = no_null_columns(category_df)
    agree_comments = category_df[category_df.columns[category_df.all()]]
    non_hateful_comments = category_df[category_df.columns[(~category_df.astype(bool)).all()]]
    no_agree = ~((category_df.sum() == 0) | (category_df.sum() == len(users)))
    
    disagree_comments = category_df[category_df.columns[no_agree]]
    # Estos son muchos! Filtrar sólo los de aquellos artículos
    # donde haya habido algún acuerdo
    
    allowed_articles = set(
        c.article_id for c in Comment.objects.filter(id__in=agree_comments.columns)
    )
    
    allowed_comments = Comment.objects.filter(
        id__in=non_hateful_comments.columns,
        article_id__in=allowed_articles,
    )
    
    #print(len(allowed_comments))
    comment_ids = set(c.id for c in allowed_comments)
    
    non_hateful_comments = non_hateful_comments[[
        c for c in non_hateful_comments.columns if c in comment_ids
    ]]
    return agree_comments, disagree_comments, non_hateful_comments


agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "MUJER", users
)

In [52]:
show_from_df(agree_comments)


Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 


  -  @usuario Hablo la catadora de pi........

Thelma Fardin contó cómo sigue la causa contra Juan Darthés: “Brasil le permite la impunidad” 


  -  @usuario De los Presos que sacan de las carceles ni una Palabra mucho feminismo y no e visto a ninguna de estas aborteras de izquierda defender los derechos de las mujeres ni criticar a Zaffaroni que defiende y apoya criminales y violadores con su teoria abolicionista.

  -  @usuario No entiendo cómo le pueden creer si a cualquiera se le mete la pija para adentro con esta

  -  @usuario Esta pendeja usó a Juan una noche de tensión sexual y después es tan más agradecida en morderle la mano a quien le dio de comer.
Tal vez Juan no la comió bien. Pasa!

  -  @usuario Thelmita si sos petera deciloooo @usuario


Busquemos dónde hay discrepancias

In [53]:
show_from_df(disagree_comments, show=True)


Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 


  -  @usuario Kuka
     - SI: ['gclerici']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gdamill', 'lprats']

  -  @usuario que se calle 
es cómplice del saqueo
hijos de puta
     - SI: ['gclerici']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gdamill', 'lprats']

  -  @usuario Pasarte de merca durante años, es una verdadera irresponsabilidad sanitaria
     - SI: ['gclerici']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gdamill', 'lprats']

  -  @usuario Es muy importante la opinión de los artistas fracasados en el ocaso de su carrera
     - SI: ['gclerici']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gdamill', 'lprats']

  -  @usuario Cuidado con esta chica que sabe mucho de medicina.
     - SI: ['gdamill']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'lprats']

  -  @usuario Jajaja cuánto le habrán pagado por declarar esta boludez?
     - SI: ['gclerici']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gdam

In [80]:
%%capture
show_from_df(non_hateful_comments)

## LGBTI


In [81]:

agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "LGBTI", users
)

In [82]:
show_from_df(agree_comments)


Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 


  -  @usuario asco

  -  @usuario 2 trolos casados por una rabina y escoltados por 2 pavorreales con la camiseta de river

  -  @usuario 🤢

  -  @usuario Aféitense, chicas! 💋

  -  @usuario ups el barba era trolis ... ups y parecía gente de bien

  -  @usuario Asco

  -  @usuario Asco

  -  @usuario La pucha, parece que son más los anormales. 
Hacia donde va la humanidad?
Que dos anormales se juntan, es noticia en un diario.

  -  @usuario Y a quién joraca le importa q estos 2 putitos se casen, encima se sacan selfies, q manga d trolos asquerosos culos rotos!! Como les gusta ventilar la mariconeada, se creen q uno está pendiente d sus vidas?!!


In [83]:
show_from_df(disagree_comments, show=True)


Escándalo en España: fallan los tests de coronavirus fabricados en China 


  -  @usuario Y la culp es de españa ?  O los putos chinos ?
     - SI: ['lprats']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'gdamill']

Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 


  -  @usuario Que cosa peluda 🦍😎
     - SI: ['gdamill']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'lprats']

  -  @usuario Completo
     - SI: ['gdamill']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'lprats']

  -  @usuario 😝q carajo nos importa
     - SI: ['gdamill']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'lprats']

  -  @usuario En ningun parrafo de los 70 libros que fueron escritos por inspiracion divina y en estos libros sagrados solamente figuran UNGIDOS los varones para el sacerdocio...!?
     - SI: ['dvalado', 'lprats']  NO: ['asilva', 'fdesanctis', 'gclerici', 'gdamill']

  -  @usuario Y que carajo me importa lo que hagan con sus C.
   

In [84]:
%%capture
show_from_df(non_hateful_comments)

## RACISMO

In [85]:

agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "RACISMO", users
)

In [86]:
show_from_df(agree_comments)


Escándalo en España: fallan los tests de coronavirus fabricados en China 


  -  @usuario Y la culp es de españa ?  O los putos chinos ?

  -  @usuario Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto

  -  @usuario CHINOS DE MIERDA
OJALÁ SE EXTINGAN

  -  @usuario Estos chinos son una mierda


In [87]:
show_from_df(disagree_comments, show=True)


Escándalo en España: fallan los tests de coronavirus fabricados en China 


  -  @usuario Chinverguenzas
     - SI: ['gdamill']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'lprats']

  -  @usuario @usuario Los cagaron otra vez
     - SI: ['lprats']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'gdamill']

  -  @usuario Estos son socios del que hizo el virus!!!!
     - SI: ['dvalado', 'gclerici']  NO: ['asilva', 'fdesanctis', 'gdamill', 'lprats']

  -  @usuario China va a tener que pagarle al mindo todo el daño causado
     - SI: ['dvalado', 'gclerici']  NO: ['asilva', 'fdesanctis', 'gdamill', 'lprats']

  -  @usuario No podía fallar!  Jajajajaja Perdón que me ria pero jamás les hubiera comprado a ellos!
     - SI: ['gdamill', 'lprats']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici']

  -  @usuario Los chinos van por todos nosotros.Necesitan espacio !!!
     - SI: ['dvalado', 'gclerici', 'gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']

  -  @usuario @usuario Per

In [88]:
%%capture
show_from_df(non_hateful_comments)

## POBREZA

In [89]:

agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "POBREZA", users
)

In [90]:
show_from_df(agree_comments)

In [91]:
show_from_df(disagree_comments, show=True)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 


  -  @usuario Que asco esos negros imundos lacras parasitos abria que bañarlos en corona virus
     - SI: ['asilva', 'dvalado', 'gdamill']  NO: ['fdesanctis', 'gclerici', 'lprats']


In [92]:
show_from_df(non_hateful_comments)

## POLITICA

In [93]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "POLITICA", users
)

In [94]:

show_from_df(agree_comments)

In [95]:
show_from_df(disagree_comments, show=True)


Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 


  -  @usuario Para decir algo público al menos deberia quitarse la máscara; así la reconocemos y sabemos seguro que fue ella quien lo dijo. Y no alguna nieta perdida, tal vez.
     - SI: ['gclerici']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gdamill', 'lprats']

  -  @usuario Mingo... Mira quien habla? Otra rata salió de su cueva.... OTRA VENDE PATRIA.... POR DIOS... CUANTOS TRAIDORES QUE HAY EN NUESTRO QUERIDA PATRIA 🇦🇷🇦🇷🇦🇷🇦🇷
     - SI: ['gclerici', 'lprats']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gdamill']

  -  @usuario Facho
     - SI: ['fdesanctis', 'gdamill', 'lprats']  NO: ['asilva', 'dvalado', 'gclerici']

  -  @usuario Kuka
     - SI: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'gdamill']  NO: ['lprats']

Escándalo en España: fallan los tests de coronavirus fabricados en China 


  -  @usuario China te manda el virus
Te cobra el test...no anda
El partido comuns=ista china si sibrevif

In [96]:
%%capture
show_from_df(non_hateful_comments)

## ASPECTO

In [97]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "ASPECTO", users
)

In [98]:
show_from_df(agree_comments)


Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 


  -  @usuario Irresponsable es q te sigas falopeando a los 60 años, vieja trola.

  -  @usuario Esta momia vive en Argentina o España?


In [99]:
show_from_df(disagree_comments, show=True)


Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 


  -  @usuario Estamos grandes???no la conocí....
     - SI: ['lprats']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'gdamill']

  -  @usuario Es muy importante la opinión de los artistas fracasados en el ocaso de su carrera
     - SI: ['gclerici']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gdamill', 'lprats']

  -  @usuario Vive?
     - SI: ['dvalado']  NO: ['asilva', 'fdesanctis', 'gclerici', 'gdamill', 'lprats']

  -  @usuario 🤮🤮🤮🤮
     - SI: ['fdesanctis']  NO: ['asilva', 'dvalado', 'gclerici', 'gdamill', 'lprats']

  -  @usuario Comerse a Fito es anti sanitario
     - SI: ['asilva']  NO: ['dvalado', 'fdesanctis', 'gclerici', 'gdamill', 'lprats']

  -  @usuario Para decir algo público al menos deberia quitarse la máscara; así la reconocemos y sabemos seguro que fue ella quien lo dijo. Y no alguna nieta perdida, tal vez.
     - SI: ['asilva', 'lprats']  NO: ['dvalado', 'fdesanctis', 'gcleri

In [74]:
%%capture
show_from_df(non_hateful_comments)

## CRIMINAL

In [100]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "CRIMINAL", users
)

In [101]:
show_from_df(agree_comments)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 


  -  @usuario Uno menos.

  -  @usuario Al fin una buena!!!

  -  @usuario Uno menos..

  -  @usuario 👏🏻👏🏻👏🏻 sigan sigan

  -  @usuario Igual no servía para nada......

  -  @usuario Uno menos 💪🏼💪🏼💪🏼

  -  @usuario Que asco esos negros imundos lacras parasitos abria que bañarlos en corona virus

  -  @usuario Loa beneficios del Covid19

  -  @usuario Deberían de aprovechar lo del coronavirus, y decir que atacó a todos los reclusos y, por eso debieron cremarlos🤷🏻‍♂️

  -  @usuario Pero miren a esta criatura de belcebú, con la cría de asesino en brazos para dar lástima. Ay! ay! Cobras y leones hambrientos" Problema solucionado.

  -  @usuario Una lacra menos 🤷🏼‍♂️

  -  @usuario Un arma tatuada en la cara... Uno menos como dicen muchos...

  -  @usuario Gracias Dios, te quedan vários todavía para llevarte.

  -  @usuario Uno menos 👏🏽👏🏽

  -  @usuario Uno menos

  -  @usuario Si se morían 1000 se solucio

In [102]:
show_from_df(disagree_comments, show=True)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 


  -  @usuario Delicatesen los tatus
     - SI: ['gdamill']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'lprats']

  -  @usuario Bueno. Sigamos con cosas mas interesantes.
     - SI: ['gdamill']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'lprats']

  -  @usuario Mira lo que es ese muñeco por favor 💀
     - SI: ['asilva', 'gdamill', 'lprats']  NO: ['dvalado', 'fdesanctis', 'gclerici']

  -  @usuario Qepd ahre
     - SI: ['dvalado', 'fdesanctis', 'gdamill']  NO: ['asilva', 'gclerici', 'lprats']

  -  @usuario Q enorme perdida
     - SI: ['fdesanctis', 'gdamill', 'lprats']  NO: ['asilva', 'dvalado', 'gclerici']

  -  @usuario Que lastima
     - SI: ['dvalado', 'fdesanctis', 'gdamill']  NO: ['asilva', 'gclerici', 'lprats']

  -  @usuario Punto a favor
     - SI: ['asilva', 'fdesanctis', 'gdamill', 'lprats']  NO: ['dvalado', 'gclerici']

  -  @usuario Pobre ciudadano, seguramente era inoc

In [103]:
%%capture
show_from_df(non_hateful_comments, show=True)

## SALUD


In [104]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "DISCAPACIDAD", users
)

In [105]:
show_from_df(agree_comments)

In [106]:
show_from_df(disagree_comments, show=True)


Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 


  -  @usuario Esta vieja está en pedo
     - SI: ['dvalado']  NO: ['asilva', 'fdesanctis', 'gclerici', 'gdamill', 'lprats']

  -  @usuario Pasarte de merca durante años, es una verdadera irresponsabilidad sanitaria
     - SI: ['dvalado', 'fdesanctis', 'gdamill']  NO: ['asilva', 'gclerici', 'lprats']

  -  @usuario Irresponsable es q te sigas falopeando a los 60 años, vieja trola.
     - SI: ['dvalado', 'fdesanctis', 'gdamill']  NO: ['asilva', 'gclerici', 'lprats']

Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 


  -  @usuario Tal vez después de eso se le pasa la histeria.
     - SI: ['gdamill']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'lprats']

  -  @usuario Lleno d Nano Cerebros minusválidos infectados con el peor d todos los Virus e Peroncho KK. Ese cuando se les mete en el cuerpo hace estragos dejándolos inservibles y llenos d fanatismo, lo pe

In [107]:
show_from_df(non_hateful_comments)

## OTROS

In [108]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "OTROS", users
)

In [109]:
show_from_df(agree_comments)

In [111]:
show_from_df(disagree_comments, show=True)


Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 


  -  @usuario Pasarte de merca durante años, es una verdadera irresponsabilidad sanitaria
     - SI: ['asilva']  NO: ['dvalado', 'fdesanctis', 'gclerici', 'gdamill', 'lprats']

  -  @usuario Es muy importante la opinión de los artistas fracasados en el ocaso de su carrera
     - SI: ['asilva']  NO: ['dvalado', 'fdesanctis', 'gclerici', 'gdamill', 'lprats']

  -  @usuario Irresponsable es q te sigas falopeando a los 60 años, vieja trola.
     - SI: ['asilva']  NO: ['dvalado', 'fdesanctis', 'gclerici', 'gdamill', 'lprats']


## LLAMADO A LA ACCIÓN

Queremos ver:

- Aquellas que hayan sido etiquetadas como odiosas


In [120]:
my_users = users
df_calls_for_action = pd.DataFrame({"name": my_users})
df_calls_for_action.set_index("name", inplace=True)

for comment_label in CommentLabel.objects.filter(is_hateful=True, article_label__user__username__in=my_users):
    username = comment_label.article_label.user.username
    df_calls_for_action.loc[username, comment_label.comment_id] = comment_label.calls_for_action

In [121]:
kappa_of(df_calls_for_action, users)

(0.6078824988991359, 52)

In [122]:
for i in range(len(df_calls_for_action)):
    user1 = df_calls_for_action.index[i]
    for j in range(i+1, len(df_calls_for_action)):
        user2 = df_calls_for_action.index[j]
        
        print(f"Kappa {user1:<10} - {user2:<10} = {kappa_of(df_calls_for_action, [user1, user2])}")

Kappa asilva     - dvalado    = (0.9513212795549376, 70)
Kappa asilva     - fdesanctis = (0.6685878962536026, 69)
Kappa asilva     - gclerici   = (0.3052631578947365, 72)
Kappa asilva     - gdamill    = (0.8279118572927596, 82)
Kappa asilva     - lprats     = (0.7748815165876779, 76)
Kappa dvalado    - fdesanctis = (0.41095890410958896, 86)
Kappa dvalado    - gclerici   = (0.2266666666666664, 87)
Kappa dvalado    - gdamill    = (0.6390570245301054, 103)
Kappa dvalado    - lprats     = (0.5720789074355085, 94)
Kappa fdesanctis - gclerici   = (0.36585365853658613, 78)
Kappa fdesanctis - gdamill    = (0.4940796555435951, 94)
Kappa fdesanctis - lprats     = (0.6666666666666669, 91)
Kappa gclerici   - gdamill    = (0.2978723404255316, 99)
Kappa gclerici   - lprats     = (0.3174395819725665, 95)
Kappa gdamill    - lprats     = (0.8459403192227621, 111)


In [123]:
df_calls_for_action.sum(axis=1) / df_calls_for_action.notna().sum(axis=1)

name
asilva        0.166667
dvalado       0.201835
fdesanctis    0.070000
gclerici      0.026316
gdamill       0.115108
lprats        0.121951
dtype: float64

In [127]:
df = no_null_columns(df_calls_for_action)

df.sum(axis=1)

name
asilva        11.0
dvalado       12.0
fdesanctis     6.0
gclerici       3.0
gdamill       14.0
lprats        11.0
dtype: float64

In [128]:

agree_comments = df[df.columns[df.all()]]
agree_comments.shape

no_agree = ~((df.sum() == 0) | (df.sum() == len(users)))
no_agree_comments = df[df.columns[no_agree]]

In [129]:
show_from_df(agree_comments)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 


  -  @usuario Que asco esos negros imundos lacras parasitos abria que bañarlos en corona virus

  -  @usuario Que no pare el fuego y rociarlo con nafta es una buena receta..


In [131]:
show_from_df(no_agree_comments, show=True)


Escándalo en España: fallan los tests de coronavirus fabricados en China 


  -  @usuario CHINOS DE MIERDA
OJALÁ SE EXTINGAN
     - SI: ['asilva', 'dvalado', 'gdamill']  NO: ['fdesanctis', 'gclerici', 'lprats']

El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 


  -  @usuario Parece que este virus nos va a depurar un poco de algunos males...
     - SI: ['gdamill']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici', 'lprats']

  -  @usuario Gracias Dios, te quedan vários todavía para llevarte.
     - SI: ['gdamill', 'lprats']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici']

  -  @usuario Si se morían 1000 se solucionaba el problema de raíz
     - SI: ['gdamill', 'lprats']  NO: ['asilva', 'dvalado', 'fdesanctis', 'gclerici']

  -  @usuario Esperemos que sea el primero pero no el último
     - SI: ['asilva', 'dvalado', 'gdamill']  NO: ['fdesanctis', 'gclerici', 'lprats']

  -  @usuario Pero si tienen k morir todos los presos
     - SI: ['asilva', '